In [2]:
import pandas as pd
import numpy as np
import requests
import os
import json
from datetime import datetime
import pandas as pd
import time  # Import time module for delay
import math
import csv
import os
print("libraries loaded")

libraries loaded


In [3]:
notebook_dir = os.getcwd()
print(notebook_dir)

C:\Users\ashle\Desktop\School\Y3S2\dse3101\Project\Backend


In [3]:
#Generate API key if expired, key in your own onemap api email and password below to run the code!

email = "enter_email"
password = "enter_password"
            
url = "https://www.onemap.gov.sg/api/auth/post/getToken"
            
payload = {
              "email": email,
              "password":password
            } 
            
response = requests.request("POST", url, json=payload)
json_response = response.json()
API_KEY = json_response["access_token"]
expiry_timestamp = int(json_response["expiry_timestamp"])
expiry_time = datetime.utcfromtimestamp(expiry_timestamp).strftime('%Y-%m-%d %H:%M')
print(f"API key retrieved! Expires on: {expiry_time}")

API key retrieved! Expires on: 2025-04-21 16:57


In [5]:
import requests
import csv

# API URL to search for MRT stations
url = "https://www.onemap.gov.sg/api/common/elastic/search"

# Search for MRT STATION by setting the searchVal to "MRT STATION"
params = {
    "searchVal": "mrt station",  
    "returnGeom": "Y",  
    "getAddrDetails": "Y", 
}

headers = {"Authorization": API_KEY}

response = requests.get(url, headers=headers, params=params)

data = response.json()
pages = int(data['totalNumPages'])

mrt_stations = []
mrt_stations_exit = []

for page in range(1, pages + 1):
    # Update params for each page
    params["pageNum"] = page
    response = requests.get(url, headers=headers, params=params)

    # Parse the response for the current page
    data = response.json()

    # Loop through each result and extract building, latitude, and longitude
    for item in data['results']:
        building = item.get("BUILDING")  # Default if no building name
        latitude = item.get("LATITUDE")  # Default if no latitude
        longitude = item.get("LONGITUDE")  # Default if no longitude
        
        if "EXIT" in building.upper():
            mrt_stations_exit.append({
            "building": building,
            "latitude": latitude,
            "longitude": longitude
        })
        else:
            # Append the extracted information to the list
            mrt_stations.append({
                "building": building,
                "latitude": latitude,
                "longitude": longitude
            })

csv_filename1 = "mrt_stations.csv"

#with open(csv_filename1, mode='w', newline='') as file:
#    writer = csv.DictWriter(file, fieldnames=["building", "latitude", "longitude"])

#    writer.writeheader()

#    writer.writerows(mrt_stations)

print(f"Data saved to {csv_filename1}")




Data saved to mrt_stations.csv


In [5]:
mrt_coordinates_df = pd.read_csv("raw_data/mrt_coordinates.csv")
print("Coordinates loaded!")
print(mrt_coordinates_df)

Coordinates loaded!
     Unnamed: 0     Station_Name   longitude  latitude
0             1        admiralty  103.800991  1.440589
1             2         aljunied  103.882906  1.316433
2             3       ang mo kio  103.849558  1.369933
3             4          bartley  103.880187  1.342473
4             5         bayfront  103.859080  1.281874
..          ...              ...         ...       ...
138         139  woodlands south  103.793264  1.427396
139         140        woodleigh  103.870890  1.339509
140         141          yew tee  103.747405  1.397535
141         142     yio chu kang  103.844947  1.381756
142         143           yishun  103.835005  1.429443

[143 rows x 4 columns]


In [71]:
# we will filter out by harversine distance

headers = {
    "Authorization": API_KEY
}
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c 

# Prepare an empty list to store results
results = []
counter = 0
# Loop through each pair of stations
for index1, row1 in mrt_coordinates_df.iterrows():
    for index2, row2 in mrt_coordinates_df.iterrows():
        # Skip if it's the same station
        if index1 >= index2:
            continue
        
        # Get the coordinates for the start and end stations
        start_lat = row1["latitude"]
        start_lon = row1["longitude"]
        end_lat = row2["latitude"]
        end_lon = row2["longitude"]
        distance_km = haversine(start_lat, start_lon, end_lat, end_lon)
        
        # Skip API request if distance is more than 1 km
        if distance_km > 1:
            continue
        
        # Construct the API URL dynamically
        url = f"https://www.onemap.gov.sg/api/public/routingsvc/route?start={start_lat}%2C{start_lon}&end={end_lat}%2C{end_lon}&routeType=walk&maxWalkDistance = 1500"
        
        # Make the API request
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            # Get the data from the response
            data = response.json()
            
            # Extract total distance (assuming it's in the route_summary part of the response)
            total_distance = data['route_summary'].get('total_distance', "N/A")
            total_time = (data['route_summary'].get('total_distance', "N/A"))
            
            # Create a dictionary for the route info
            route_info = {
                "Start_Station": row1["Station_Name"],
                "End_Station": row2["Station_Name"],
                "Total_Distance_m": total_distance,  # Only appending the total distance
                "Total_Time_minutes" : total_time//60
            }
            results.append(route_info)
        else:
            print(f"Failed to get route between {row1['Station_Name']} and {row2['Station_Name']}, status code {response.status_code}")
        counter+=1
        if counter%250 == 0:
            time.sleep(60)

# Convert the results into a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
#results_df.to_csv("mrt_walkability_distances.csv", index=False)
results_df.head()
print("Walkability distances saved to mrt_walkability_distances.csv")


Walkability distances saved to mrt_walkability_distances.csv


In [76]:
# Define the headers for the API request
headers = {
    "Authorization": API_KEY
}
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c 



mrt_stations_df = pd.DataFrame(mrt_stations)
# print(mrt_stations_df.head())

with open("bus_stops.json", 'r') as f:
    bus_stops = json.load(f)
    
    
bus_stops = pd.DataFrame(bus_stops)

# Prepare an empty list to store results
nearby_busstops = []
counter = 0 # measure to make sure we do not exceed the api call limit
# Loop through each pair of stations
for index1, row1 in mrt_coordinates_df.iterrows():
    for index2, row2 in bus_stops.iterrows():
        
        # Get the coordinates for the start and end stations
        start_lat = float(row1["latitude"])
        start_lon = float(row1["longitude"])
        end_lat = float(row2["Latitude"])
        end_lon = float(row2["Longitude"])
        distance_km = haversine(start_lat, start_lon, end_lat, end_lon)
        
        # Skip API request if distance is more than .5 km
        if distance_km > 0.5:
            continue
        
        # Construct the API URL dynamically
        url = f"https://www.onemap.gov.sg/api/public/routingsvc/route?start={start_lat}%2C{start_lon}&end={end_lat}%2C{end_lon}&routeType=walk&maxWalkDistance=500"
        
        # Make the API request
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            # Get the data from the response
            data = response.json()
            
            # Extract total distance (assuming it's in the route_summary part of the response)
            total_distance = data['route_summary'].get('total_distance', "N/A")
            total_time = (data['route_summary'].get('total_distance', "N/A"))
            
            # Create a dictionary for the route info
            route_info = {
                "MRT station": row1["Station_Name"],
                "Bus stop": str(row2["BusStopCode"]),
                "Total_Distance_m": total_distance,  # Only appending the total distance
                "Total_Time_minutes" : total_time//60
            }
            results.append(route_info)
        else:
            print(f"Failed to get route between {row1['Station_Name']} and {row2['BusStopCode']}, status code {response.status_code}")
        counter+=1
        if counter%250 == 0:
            time.sleep(60)

# Convert the results into a DataFrame
results_df = pd.DataFrame(results)





Failed to get route between changi airport and 95129, status code 404


In [78]:
#results_df.to_csv("nearbybusstops.csv", index=False)
print("nearbybusstops saved to nearbybusstops.csv")

nearbybusstops saved to nearbybusstops.csv
